CONFIGURACIÓN Y CONEXIÓN

In [ ]:
import duckdb
import os
# Ruta al archivo de la base de datos DuckDB
DB_PATH = "lakehouse/mobility.duckdb"
DATA_PATH = "data/raw"

con = duckdb.connect(DB_PATH)

print("Conectado a:", DB_PATH)

# Comprobar que el archivo existe dentro de lakehouse/
print("Contenido de la carpeta lakehouse:", os.listdir("lakehouse"))

Conectado a: lakehouse/mobility.duckdb
Contenido de la carpeta lakehouse: ['mobility.duckdb']


In [6]:
# Tablas BRONZE (raw / staging)

q("""
CREATE TABLE IF NOT EXISTS bronze_mitma_od_daily (
    date             DATE,
    zone_level       VARCHAR,   -- 'district', 'municipal', 'gau'
    origin_zone_id   INTEGER,
    destination_zone_id INTEGER,
    hour             INTEGER,   -- 0-23
    trips            BIGINT,
    data_source      VARCHAR    -- por si luego mezclas fuentes
);
""")

q("""
CREATE TABLE IF NOT EXISTS bronze_ine_population (
    year             INTEGER,
    zone_level       VARCHAR,
    zone_id          INTEGER,
    population_total BIGINT
);
""")

# Tablas SILVER (limpias / integradas)

q("""
CREATE TABLE IF NOT EXISTS silver_od_clean (
    date             DATE,
    zone_level       VARCHAR,
    origin_zone_id   INTEGER,
    destination_zone_id INTEGER,
    hour             INTEGER,
    trips            DOUBLE,
    population_origin BIGINT,
    population_destination BIGINT
);
""")

# Tablas GOLD (agregados para análisis)

q("""
CREATE TABLE IF NOT EXISTS gold_typical_day_patterns (
    zone_level       VARCHAR,
    origin_zone_id   INTEGER,
    destination_zone_id INTEGER,
    hour             INTEGER,
    avg_trips        DOUBLE,
    pattern_type     VARCHAR    -- ej. 'weekday', 'weekend', 'holiday'
);
""")

,Count


In [7]:
q("PRAGMA show_tables;")

,name
0,bronze_ine_population
1,bronze_mitma_od_daily
2,gold_typical_day_patterns
3,silver_od_clean


CONFIGURACIÓN Y CONEXIÓN REAL

In [7]:
import duckdb
import pandas as pd
from textwrap import dedent

DB_PATH = "lakehouse/mobility.duckdb"
DATA_PATH = "data/raw"  # aquí  luego los CSV

con = duckdb.connect(DB_PATH)

def q(sql: str, **params) -> pd.DataFrame:
    """Ejecuta SQL y devuelve un DataFrame de pandas."""
    return con.execute(dedent(sql), params).fetchdf()

print("DuckDB:", con.execute("SELECT version();").fetchone()[0])

DuckDB: v1.4.2


1º Leer CSV bruto (municipios)

In [8]:
q("""
SELECT *
FROM read_csv_auto('data/raw/municipal_202301/20230101_Viajes_municipios.csv.gz', compression='gzip', header=True)
LIMIT 20;
""")

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,estudio_origen_posible,estudio_destino_posible,residencia,renta,edad,sexo,viajes,viajes_km
0,20230101,00,01001,01001,2-10,casa,frecuente,False,False,01,10-15,NA,NA,2.922000,12.806000
1,20230101,00,01001,01001,2-10,frecuente,casa,False,False,01,10-15,NA,NA,5.202000,22.098000
2,20230101,00,01001,01009_AM,0.5-2,frecuente,casa,False,False,01,>15,NA,NA,1.000000,1.000000
3,20230101,00,01001,01009_AM,0.5-2,no_frecuente,no_frecuente,False,False,01,10-15,NA,NA,2.336398,2.657702
4,20230101,00,01001,01009_AM,2-10,casa,frecuente,False,False,01,10-15,NA,NA,2.354009,13.755398
5,20230101,00,01001,01009_AM,2-10,frecuente,casa,False,False,01,>15,NA,NA,1.267995,3.270958
6,20230101,00,01001,01009_AM,2-10,no_frecuente,casa,False,False,01,>15,NA,NA,1.000000,2.021356
7,20230101,00,01001,01009_AM,2-10,no_frecuente,no_frecuente,False,False,48,>15,65-100,hombre,1.186984,4.218432
8,20230101,00,01001,01017_AM,10-50,frecuente,casa,False,False,01,10-15,NA,NA,6.000000,109.772000
9,20230101,00,01001,01054_AM,10-50,no_frecuente,casa,False,False,01,10-15,NA,NA,3.804000,127.268000


PASAR A BRONZE

borro una que tenia antes para poder crear la del proyecto real

In [9]:
q("DROP TABLE IF EXISTS bronze_mitma_od_daily;")

,Success


In [10]:
q("""
CREATE TABLE bronze_mitma_od_daily (
    fecha TEXT,
    periodo TEXT,
    origen TEXT,
    destino TEXT,
    distancia TEXT,
    actividad_origen TEXT,
    actividad_destino TEXT,
    estudio_origen_posible TEXT,
    estudio_destino_posible TEXT,
    residencia TEXT,
    renta TEXT,
    edad TEXT,
    sexo TEXT,
    viajes DOUBLE,
    viajes_km DOUBLE,
    data_source TEXT
);
""")

,Count


SOLUCION

Inserto todos los archivos de un mes de "Municipios"

In [12]:
import glob

files = sorted(glob.glob("data/raw/municipal_202301/*.csv.gz"))
print("Total CSVs encontrados:", len(files))

for f in files:
    print("Cargando:", f)

    q(f"""
        INSERT INTO bronze_mitma_od_daily
        SELECT
            fecha,                         -- raw
            periodo,                       -- raw
            origen,
            destino,
            distancia,
            actividad_origen,
            actividad_destino,
            estudio_origen_posible,
            estudio_destino_posible,
            residencia,
            renta,
            edad,
            sexo,
            viajes,
            viajes_km,
            'MITMA_202301' AS data_source
        FROM read_csv_auto('{f}', header=True, compression='gzip', all_varchar=True)
    """)

Total CSVs encontrados: 31
Cargando: data/raw/municipal_202301/20230101_Viajes_municipios.csv.gz


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cargando: data/raw/municipal_202301/20230102_Viajes_municipios.csv.gz


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cargando: data/raw/municipal_202301/20230103_Viajes_municipios.csv.gz


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cargando: data/raw/municipal_202301/20230130_Viajes_municipios.csv.gz


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cargando: data/raw/municipal_202301/20230131_Viajes_municipios.csv.gz


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Ejecuta de nuevo la ingesta Bronze con all_varchar=True.

*total de registros cargados en Bronze.

In [13]:
q("SELECT COUNT(*) FROM bronze_mitma_od_daily;")

,count_star()
0,362933253


In [14]:
q("SELECT * FROM bronze_mitma_od_daily LIMIT 31;")


,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,estudio_origen_posible,estudio_destino_posible,residencia,renta,edad,sexo,viajes,viajes_km,data_source
0,20230101,00,01001,01001,2-10,casa,frecuente,no,no,01,10-15,NA,NA,2.922000,12.806000,MITMA_202301
1,20230101,00,01001,01001,2-10,frecuente,casa,no,no,01,10-15,NA,NA,5.202000,22.098000,MITMA_202301
2,20230101,00,01001,01009_AM,0.5-2,frecuente,casa,no,no,01,>15,NA,NA,1.000000,1.000000,MITMA_202301
3,20230101,00,01001,01009_AM,0.5-2,no_frecuente,no_frecuente,no,no,01,10-15,NA,NA,2.336398,2.657702,MITMA_202301
4,20230101,00,01001,01009_AM,2-10,casa,frecuente,no,no,01,10-15,NA,NA,2.354009,13.755398,MITMA_202301
5,20230101,00,01001,01009_AM,2-10,frecuente,casa,no,no,01,>15,NA,NA,1.267995,3.270958,MITMA_202301
6,20230101,00,01001,01009_AM,2-10,no_frecuente,casa,no,no,01,>15,NA,NA,1.000000,2.021356,MITMA_202301
7,20230101,00,01001,01009_AM,2-10,no_frecuente,no_frecuente,no,no,48,>15,65-100,hombre,1.186984,4.218432,MITMA_202301
8,20230101,00,01001,01017_AM,10-50,frecuente,casa,no,no,01,10-15,NA,NA,6.000000,109.772000,MITMA_202301
9,20230101,00,01001,01054_AM,10-50,no_frecuente,casa,no,no,01,10-15,NA,NA,3.804000,127.268000,MITMA_202301


                                             SILVER

In [ ]:
q("DROP TABLE IF EXISTS silver_od_clean;")

In [ ]:
q("""
CREATE TABLE silver_od_clean AS
SELECT
    -- 1. Limpieza de fecha: YYYY-MM-DD -> DATE 
    CAST(SUBSTR(fecha,1,4) || '-' || SUBSTR(fecha,5,2) || '-' || SUBSTR(fecha,7,2) AS DATE)
        AS date,

    -- 2. Hora limpia (MITMA la llama "periodo")
    CAST(periodo AS INTEGER) AS hour,

    -- 3. Origen y destino: separar casos con "_AM"
    origen AS origin_raw,
    destino AS destination_raw,

    -- Extra: origen limpio sin "_AM"
    NULLIF(SPLIT_PART(origen, '_', 1), '') AS origin_zone_id,

    -- Extra: destino limpio sin "_AM"
    NULLIF(SPLIT_PART(destino, '_', 1), '') AS destination_zone_id,

    -- 4. Marcar si es subzona AM o no
    CASE WHEN origen LIKE '%_AM' THEN TRUE ELSE FALSE END AS origin_is_AM,
    CASE WHEN destino LIKE '%_AM' THEN TRUE ELSE FALSE END AS destination_is_AM,

    -- 5. Distancia
    distancia AS distance_range,

    -- 6. Actividades
    actividad_origen AS origin_activity,
    actividad_destino AS destination_activity,

    -- 7. Limpieza booleans 
    CASE
        WHEN LOWER(estudio_origen_posible) IN ('si', 'true', 'c') THEN TRUE
        WHEN LOWER(estudio_origen_posible) IN ('no', 'false', 'n') THEN FALSE
        ELSE NULL
    END AS origin_study_possible,

    CASE
        WHEN LOWER(estudio_destino_posible) IN ('si', 'true', 'c') THEN TRUE
        WHEN LOWER(estudio_destino_posible) IN ('no', 'false', 'n') THEN FALSE
        ELSE NULL
    END AS destination_study_possible,

    -- 8. Limpieza de categorías (convierte NA en NULL)
    NULLIF(edad, 'NA') AS age_range,
    NULLIF(sexo, 'NA') AS gender,
    NULLIF(renta, 'NA') AS income_range,

    -- 9. Métricas principales
    CAST(viajes AS DOUBLE) AS trips,
    CAST(viajes_km AS DOUBLE) AS trips_km,

    -- 10. Fuente del dato
    data_source

FROM bronze_mitma_od_daily;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,342365963


Validar si SILVER se ha hecho correctamente

In [33]:
q("SELECT COUNT(*) FROM silver_od_clean;")

,count_star()
0,342365963


In [34]:
q("SELECT * FROM silver_od_clean LIMIT 20;")

,date,hour,origin_raw,destination_raw,origin_zone_id,destination_zone_id,origin_is_AM,destination_is_AM,distance_range,origin_activity,destination_activity,origin_study_possible,destination_study_possible,age_range,gender,income_range,trips,trips_km,data_source
0,2023-01-01,0,01001,01001,01001,01001,False,False,2-10,casa,frecuente,False,False,None,None,10-15,2.922000,12.806000,MITMA_202301
1,2023-01-01,0,01001,01001,01001,01001,False,False,2-10,frecuente,casa,False,False,None,None,10-15,5.202000,22.098000,MITMA_202301
2,2023-01-01,0,01001,01009_AM,01001,01009,False,True,0.5-2,frecuente,casa,False,False,None,None,>15,1.000000,1.000000,MITMA_202301
3,2023-01-01,0,01001,01009_AM,01001,01009,False,True,0.5-2,no_frecuente,no_frecuente,False,False,None,None,10-15,2.336398,2.657702,MITMA_202301
4,2023-01-01,0,01001,01009_AM,01001,01009,False,True,2-10,casa,frecuente,False,False,None,None,10-15,2.354009,13.755398,MITMA_202301
5,2023-01-01,0,01001,01009_AM,01001,01009,False,True,2-10,frecuente,casa,False,False,None,None,>15,1.267995,3.270958,MITMA_202301
6,2023-01-01,0,01001,01009_AM,01001,01009,False,True,2-10,no_frecuente,casa,False,False,None,None,>15,1.000000,2.021356,MITMA_202301
7,2023-01-01,0,01001,01009_AM,01001,01009,False,True,2-10,no_frecuente,no_frecuente,False,False,65-100,hombre,>15,1.186984,4.218432,MITMA_202301
8,2023-01-01,0,01001,01017_AM,01001,01017,False,True,10-50,frecuente,casa,False,False,None,None,10-15,6.000000,109.772000,MITMA_202301
9,2023-01-01,0,01001,01054_AM,01001,01054,False,True,10-50,no_frecuente,casa,False,False,None,None,10-15,3.804000,127.268000,MITMA_202301


                                            GOLD

In [ ]:
q("DROP TABLE IF EXISTS gold_typical_day_hourly;")

Esto responde a Bussines question1: Typical Day. “¿Cómo se comporta un día típico de movilidad?”

In [ ]:
q("""
CREATE TABLE gold_typical_day_hourly AS
SELECT
    origin_zone_id,
    destination_zone_id,
    hour,
    AVG(trips) AS avg_trips, --viajes promedio
    SUM(trips) AS total_trips, --volumen total de viajes en el mes
    SUM(trips_km) AS total_trips_km --km recorridos totales
FROM silver_od_clean
GROUP BY origin_zone_id, destination_zone_id, hour;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,7589863


                                        WEEKND VS WEEKDAY

✔ Comparar movilidad entre días laborables y fines de semana

In [ ]:
q("DROP TABLE IF EXISTS gold_weekday_weekend;")

q("""
CREATE TABLE gold_weekday_weekend AS
SELECT
    origin_zone_id,
    destination_zone_id,
    EXTRACT(DOW FROM date) AS weekday,
    SUM(trips) AS trips
FROM silver_od_clean
GROUP BY origin_zone_id, destination_zone_id, weekday;
""")

,Count
0,4314570


MATRIX = El peso total de cada flujo OD - “¿Cuántos viajes totales hay entre cada par de zonas?”

In [37]:
q("DROP TABLE IF EXISTS gold_od_matrix;")

q("""
CREATE TABLE gold_od_matrix AS
SELECT
    origin_zone_id,
    destination_zone_id,
    SUM(trips) AS total_trips,
    SUM(trips_km) AS total_trips_km
FROM silver_od_clean
GROUP BY origin_zone_id, destination_zone_id;
""")

,Count
0,1468076


 Exportaciones Lakehouse

In [38]:
q("""
COPY silver_od_clean
TO 'data/silver/municipal_202301_clean.parquet'
(FORMAT PARQUET);
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,342365963


In [39]:
q("""
COPY gold_typical_day_hourly
TO 'data/gold/typical_day_hourly.parquet'
(FORMAT PARQUET);
""")

,Count
0,7589863


In [40]:
q("""
COPY gold_od_matrix
TO 'data/gold/od_matrix.parquet'
(FORMAT PARQUET);
""")

,Count
0,1468076


                                    DISTRITOS

In [43]:
q("DROP TABLE IF EXISTS bronze_mitma_od_distritos;")
q("""
CREATE TABLE bronze_mitma_od_distritos (
    fecha TEXT,
    periodo TEXT,
    origen TEXT,
    destino TEXT,
    distancia TEXT,
    actividad_origen TEXT,
    actividad_destino TEXT,
    estudio_origen_posible TEXT,
    estudio_destino_posible TEXT,
    residencia TEXT,
    renta TEXT,
    edad TEXT,
    sexo TEXT,
    viajes DOUBLE,
    viajes_km DOUBLE,
    data_source TEXT
);
""")

,Count


                        Cargar los CSV.gz sin interpretar tipos

In [44]:
import glob

files = sorted(glob.glob("data/raw/distritos_202301/*.csv.gz"))
print("Archivos distrito:", len(files))

for f in files:
    print("Cargando:", f)

    q(f"""
        INSERT INTO bronze_mitma_od_distrito
        SELECT
            fecha,
            periodo,
            origen,
            destino,
            distancia,
            actividad_origen,
            actividad_destino,
            estudio_origen_posible,
            estudio_destino_posible,
            residencia,
            renta,
            edad,
            sexo,
            viajes,
            viajes_km,
            'DISTRITO_202301'
        FROM read_csv_auto(
            '{f}',
            header=True,
            compression='gzip',
            all_varchar=True
        );
    """)

Archivos distrito: 31
Cargando: data/raw/distritos_202301/20230101_Viajes_distritos.csv.gz


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cargando: data/raw/distritos_202301/20230102_Viajes_distritos.csv.gz


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cargando: data/raw/distritos_202301/20230103_Viajes_distritos.csv.gz


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cargando: data/raw/distritos_202301/20230130_Viajes_distritos.csv.gz


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cargando: data/raw/distritos_202301/20230131_Viajes_distritos.csv.gz


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [45]:
q("SELECT COUNT(*) FROM bronze_mitma_od_distrito;")

,count_star()
0,552509346


                                    
                                        SILVER

In [46]:
q("DROP TABLE IF EXISTS silver_mitma_od_distrito;")

q("""
CREATE TABLE silver_mitma_od_distrito AS
SELECT
    CAST(SUBSTR(fecha,1,4) || '-' || SUBSTR(fecha,5,2) || '-' || SUBSTR(fecha,7,2) AS DATE) AS date,
    CAST(periodo AS INTEGER) AS hour,

    origen AS origin_raw,
    destino AS destination_raw,

    NULLIF(SPLIT_PART(origen, '_', 1), '') AS origin_zone_id,
    NULLIF(SPLIT_PART(destino, '_', 1), '') AS destination_zone_id,

    origen LIKE '%_AM' AS origin_is_AM,
    destino LIKE '%_AM' AS destination_is_AM,

    distancia AS distance_range,
    actividad_origen AS origin_activity,
    actividad_destino AS destination_activity,

    CASE
        WHEN LOWER(estudio_origen_posible) IN ('si','true','c') THEN TRUE
        WHEN LOWER(estudio_origen_posible) IN ('no','false','n') THEN FALSE
        ELSE NULL
    END AS origin_study_possible,

    CASE
        WHEN LOWER(estudio_destino_posible) IN ('si','true','c') THEN TRUE
        WHEN LOWER(estudio_destino_posible) IN ('no','false','n') THEN FALSE
        ELSE NULL
    END AS destination_study_possible,

    NULLIF(edad, 'NA') AS age_range,
    NULLIF(sexo, 'NA') AS gender,
    NULLIF(renta, 'NA') AS income_range,

    CAST(viajes AS DOUBLE) AS trips,
    CAST(viajes_km AS DOUBLE) AS trips_km,

    data_source

FROM bronze_mitma_od_distrito;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,552509346


In [47]:
q("SELECT COUNT(*) FROM silver_mitma_od_distrito;")

,count_star()
0,552509346


In [48]:
q("SELECT * FROM silver_mitma_od_distrito LIMIT 20;")

,date,hour,origin_raw,destination_raw,origin_zone_id,destination_zone_id,origin_is_AM,destination_is_AM,distance_range,origin_activity,destination_activity,origin_study_possible,destination_study_possible,age_range,gender,income_range,trips,trips_km,data_source
0,2023-01-01,0,01001,01001,01001,01001,False,False,2-10,casa,frecuente,False,False,None,None,10-15,2.922,12.806,DISTRITO_202301
1,2023-01-01,0,01001,01001,01001,01001,False,False,2-10,frecuente,casa,False,False,None,None,10-15,5.202,22.098,DISTRITO_202301
2,2023-01-01,1,01001,01001,01001,01001,False,False,2-10,no_frecuente,no_frecuente,False,False,25-45,None,>15,4.648,20.805,DISTRITO_202301
3,2023-01-01,2,01001,01001,01001,01001,False,False,2-10,casa,frecuente,False,False,None,None,10-15,2.280,9.213,DISTRITO_202301
4,2023-01-01,3,01001,01001,01001,01001,False,False,2-10,frecuente,casa,False,False,None,None,10-15,2.922,11.720,DISTRITO_202301
5,2023-01-01,3,01001,01001,01001,01001,False,False,2-10,trabajo_estudio,casa,False,False,None,None,10-15,7.219,28.876,DISTRITO_202301
6,2023-01-01,7,01001,01001,01001,01001,False,False,2-10,casa,frecuente,False,False,None,None,10-15,2.922,11.720,DISTRITO_202301
7,2023-01-01,7,01001,01001,01001,01001,False,False,2-10,casa,trabajo_estudio,False,False,None,None,10-15,2.881,10.910,DISTRITO_202301
8,2023-01-01,8,01001,01001,01001,01001,False,False,2-10,frecuente,frecuente,False,False,45-65,mujer,>15,8.183,32.712,DISTRITO_202301
9,2023-01-01,9,01001,01001,01001,01001,False,False,2-10,casa,trabajo_estudio,False,False,None,None,10-15,3.134,13.079,DISTRITO_202301


GOLD Typical Day

In [49]:
q("DROP TABLE IF EXISTS gold_typical_day_distrito;")

q("""
CREATE TABLE gold_typical_day_distrito AS
SELECT
    origin_zone_id,
    destination_zone_id,
    hour,
    AVG(trips) AS avg_trips,
    SUM(trips) AS total_trips
FROM silver_mitma_od_distrito
GROUP BY origin_zone_id, destination_zone_id, hour;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,14657782


GOLD OD Matrix

In [50]:
q("DROP TABLE IF EXISTS gold_od_matrix_distrito;")

q("""
CREATE TABLE gold_od_matrix_distrito AS
SELECT
    origin_zone_id,
    destination_zone_id,
    SUM(trips) AS total_trips
FROM silver_mitma_od_distrito
GROUP BY origin_zone_id, destination_zone_id;
""")

,Count
0,2843537


GAU BRONZE

In [ ]:
q("DROP TABLE IF EXISTS bronze_mitma_od_gau;")

q("""
CREATE TABLE bronze_mitma_od_gau (
    fecha TEXT,
    periodo TEXT,
    origen TEXT,
    destino TEXT,
    distancia TEXT,
    actividad_origen TEXT,
    actividad_destino TEXT,
    estudio_origen_posible TEXT,
    estudio_destino_posible TEXT,
    residencia TEXT,
    renta TEXT,
    edad TEXT,
    sexo TEXT,
    viajes DOUBLE,
    viajes_km DOUBLE,
    data_source TEXT
);
""")

import glob
files = sorted(glob.glob("data/raw/gau_202301/*.csv.gz"))
print("Archivos GAU:", len(files))

for f in files:
    print("Cargando:", f)
    q(f"""
        INSERT INTO bronze_mitma_od_gau
        SELECT
            fecha,
            periodo,
            origen,
            destino,
            distancia,
            actividad_origen,
            actividad_destino,
            estudio_origen_posible,
            estudio_destino_posible,
            residencia,
            renta,
            edad,
            sexo,
            viajes,
            viajes_km,
            'GAU_202301'
        FROM read_csv_auto('{f}', header=True, compression='gzip', all_varchar=True);
    """)

Archivos GAU: 31
Cargando: data/raw/gau_202301/20230101_Viajes_GAU.csv.gz


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cargando: data/raw/gau_202301/20230102_Viajes_GAU.csv.gz


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cargando: data/raw/gau_202301/20230103_Viajes_GAU.csv.gz


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cargando: data/raw/gau_202301/20230130_Viajes_GAU.csv.gz


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cargando: data/raw/gau_202301/20230131_Viajes_GAU.csv.gz


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

validar bronze

In [52]:
q("SELECT * FROM bronze_mitma_od_gau LIMIT 10;")

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,estudio_origen_posible,estudio_destino_posible,residencia,renta,edad,sexo,viajes,viajes_km,data_source
0,20230101,00,01001,01001,2-10,casa,frecuente,no,no,01,10-15,NA,NA,2.922000,12.806000,GAU_202301
1,20230101,00,01001,01001,2-10,frecuente,casa,no,no,01,10-15,NA,NA,5.202000,22.098000,GAU_202301
2,20230101,00,01001,01009_AM,0.5-2,frecuente,casa,no,no,01,>15,NA,NA,1.000000,1.000000,GAU_202301
3,20230101,00,01001,01009_AM,0.5-2,no_frecuente,no_frecuente,no,no,01,10-15,NA,NA,2.336398,2.657702,GAU_202301
4,20230101,00,01001,01009_AM,2-10,casa,frecuente,no,no,01,10-15,NA,NA,2.354009,13.755398,GAU_202301
5,20230101,00,01001,01009_AM,2-10,frecuente,casa,no,no,01,>15,NA,NA,1.267995,3.270958,GAU_202301
6,20230101,00,01001,01009_AM,2-10,no_frecuente,casa,no,no,01,>15,NA,NA,1.000000,2.021356,GAU_202301
7,20230101,00,01001,01009_AM,2-10,no_frecuente,no_frecuente,no,no,48,>15,65-100,hombre,1.186984,4.218432,GAU_202301
8,20230101,00,01001,01017_AM,10-50,frecuente,casa,no,no,01,10-15,NA,NA,6.000000,109.772000,GAU_202301
9,20230101,00,01001,01054_AM,10-50,no_frecuente,casa,no,no,01,10-15,NA,NA,3.804000,127.268000,GAU_202301


silver

In [53]:
q("DROP TABLE IF EXISTS silver_mitma_od_gau;")

q("""
CREATE TABLE silver_mitma_od_gau AS
SELECT
    CAST(SUBSTR(fecha,1,4) || '-' || SUBSTR(fecha,5,2) || '-' || SUBSTR(fecha,7,2) AS DATE) AS date,
    CAST(periodo AS INTEGER) AS hour,
    origen AS origin_raw,
    destino AS destination_raw,

    NULLIF(SPLIT_PART(origen, '_', 1), '') AS origin_zone_id,
    NULLIF(SPLIT_PART(destino, '_', 1), '') AS destination_zone_id,

    origen LIKE '%_AM' AS origin_is_AM,
    destino LIKE '%_AM' AS destination_is_AM,

    distancia,
    actividad_origen,
    actividad_destino,

    CASE
        WHEN LOWER(estudio_origen_posible) IN ('si','true','c') THEN TRUE
        WHEN LOWER(estudio_origen_posible) IN ('no','false','n') THEN FALSE
        ELSE NULL
    END AS origin_study_possible,

    CASE
        WHEN LOWER(estudio_destino_posible) IN ('si','true','c') THEN TRUE
        WHEN LOWER(estudio_destino_posible) IN ('no','false','n') THEN FALSE
        ELSE NULL
    END AS destination_study_possible,

    NULLIF(edad, 'NA') AS age_range,
    NULLIF(sexo, 'NA') AS gender,
    NULLIF(renta, 'NA') AS income_range,

    CAST(viajes AS DOUBLE) AS trips,
    CAST(viajes_km AS DOUBLE) AS trips_km,

    data_source

FROM bronze_mitma_od_gau;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,205749250


validar silver

In [54]:
q("SELECT * FROM silver_mitma_od_gau LIMIT 20;")

,date,hour,origin_raw,destination_raw,origin_zone_id,destination_zone_id,origin_is_AM,destination_is_AM,distancia,actividad_origen,actividad_destino,origin_study_possible,destination_study_possible,age_range,gender,income_range,trips,trips_km,data_source
0,2023-01-01,0,01001,01001,01001,01001,False,False,2-10,casa,frecuente,False,False,None,None,10-15,2.922000,12.806000,GAU_202301
1,2023-01-01,0,01001,01001,01001,01001,False,False,2-10,frecuente,casa,False,False,None,None,10-15,5.202000,22.098000,GAU_202301
2,2023-01-01,0,01001,01009_AM,01001,01009,False,True,0.5-2,frecuente,casa,False,False,None,None,>15,1.000000,1.000000,GAU_202301
3,2023-01-01,0,01001,01009_AM,01001,01009,False,True,0.5-2,no_frecuente,no_frecuente,False,False,None,None,10-15,2.336398,2.657702,GAU_202301
4,2023-01-01,0,01001,01009_AM,01001,01009,False,True,2-10,casa,frecuente,False,False,None,None,10-15,2.354009,13.755398,GAU_202301
5,2023-01-01,0,01001,01009_AM,01001,01009,False,True,2-10,frecuente,casa,False,False,None,None,>15,1.267995,3.270958,GAU_202301
6,2023-01-01,0,01001,01009_AM,01001,01009,False,True,2-10,no_frecuente,casa,False,False,None,None,>15,1.000000,2.021356,GAU_202301
7,2023-01-01,0,01001,01009_AM,01001,01009,False,True,2-10,no_frecuente,no_frecuente,False,False,65-100,hombre,>15,1.186984,4.218432,GAU_202301
8,2023-01-01,0,01001,01017_AM,01001,01017,False,True,10-50,frecuente,casa,False,False,None,None,10-15,6.000000,109.772000,GAU_202301
9,2023-01-01,0,01001,01054_AM,01001,01054,False,True,10-50,no_frecuente,casa,False,False,None,None,10-15,3.804000,127.268000,GAU_202301


gold

In [55]:
q("DROP TABLE IF EXISTS gold_typical_day_gau;")

q("""
CREATE TABLE gold_typical_day_gau AS
SELECT
    origin_zone_id,
    destination_zone_id,
    hour,
    AVG(trips) AS avg_trips,
    SUM(trips) AS total_trips
FROM silver_mitma_od_gau
GROUP BY origin_zone_id, destination_zone_id, hour;
""")

,Count
0,4177072


In [56]:
q("DROP TABLE IF EXISTS gold_od_matrix_gau;")

q("""
CREATE TABLE gold_od_matrix_gau AS
SELECT
    origin_zone_id,
    destination_zone_id,
    SUM(trips) AS total_trips
FROM silver_mitma_od_gau
GROUP BY origin_zone_id, destination_zone_id;
""")

,Count
0,832383
